In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib widget
from cycler import K
from common_import import *


def check_config_experiment(whitelist_checkpoint):
    user = getpass.getuser()
    motionbert_root = f'/home/{user}/codes/MotionBERT'
    for checkpoint in whitelist_checkpoint:
        experiment_path = f'{motionbert_root}/experiments/{checkpoint}.txt'
        config_path = f'{motionbert_root}/configs/pose3d/{checkpoint}.yaml'
        print(f'{checkpoint}')
        if os.path.exists(config_path):
            print(f'Config# path: {config_path}')
        else:
            print(f'[No config path] {config_path}')
            with open(config_path, 'w') as f: pass
        if os.path.exists(experiment_path): print(f'Experiment path: {experiment_path}')
        else:
            print(f'[No experiment path] {experiment_path}')
            with open(experiment_path, 'w') as f: pass

def update_result_dict_pkl(blacklist_checkpoint=[]):
    user = getpass.getuser()
    if os.path.exists('result_dict.pkl'):
        print('Loading result_dict.pkl')
        result_dict = readpkl('result_dict.pkl')
    else:
        result_dict = {}
    result_root = f'/home/{user}/codes/MotionBERT/saved_results'
    checkpoint_list = os.listdir(result_root)
    for checkpoint in tqdm(checkpoint_list):
        if checkpoint in blacklist_checkpoint: continue # skip
        if checkpoint not in result_dict.keys(): result_dict[checkpoint] = {} # create new key if not exists
        checkpoint_root = os.path.join(result_root, checkpoint)
        subset_list = glob(checkpoint_root+'/*.pkl')
        for item in subset_list:
            subset = os.path.basename(item).split('.pkl')[0]
            if subset in result_dict[checkpoint].keys():
                #print(f'{subset} exists')
                continue
            print(item)
            result = readpkl(item)
            try:
                result_dict[checkpoint][subset] = {'e1': result['e1'], 'e2': result['e2']}
            except:
                print(f'Error in reading {item}')

        # remove key if not in subset_list
        for key in list(result_dict[checkpoint].keys()):
            if key not in [os.path.basename(item).split('.pkl')[0] for item in subset_list]:
                del result_dict[checkpoint][key]

    savepkl(result_dict, 'result_dict.pkl')


### Evaluate single experiment

In [2]:
checkpoint_name = 'MB_train_h36m_gt_cam_no_factor_input_centering_seed2'
check_config_experiment([checkpoint_name])

MB_train_h36m_gt_cam_no_factor_input_centering_seed2
Experiment path: /home/hrai/codes/MotionBERT/experiments/MB_train_h36m_gt_cam_no_factor_input_centering_seed2.txt
Config# path: /home/hrai/codes/MotionBERT/configs/pose3d/MB_train_h36m_gt_cam_no_factor_input_centering_seed2.yaml


In [13]:
# load model
args, opts = get_opt_args_from_model_name(checkpoint_name, verbose=False)
model_pos, chk_filename, checkpoint = load_model(opts, args)
# load dataset
subset = 'FIT3D-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_REVOLUTE-TR_S03'
subset_path = f'/home/{user}/codes/MotionBERT/data/motion3d/yaml_files/{subset}.yaml'
print(subset_path)
if 'PCL' in subset: args.fix_orientation_pred = True # load dataset
args.subset_list = [subset]
train_loader, test_loader, _, _, datareader = load_dataset(args, use_new_datareader=True)
e1, e2, results_all, inputs_all, gts_all, total_result_dict = evaluate(args, model_pos, test_loader, datareader, checkpoint, only_one_batch=False)

# save results
save = True
if save:
    save_folder = f'/home/hrai/codes/MotionBERT/saved_results/{checkpoint_name}'
    if not os.path.exists(save_folder): os.makedirs(save_folder)
    save_path = os.path.join(save_folder, f'{subset}.pkl')
    results_dict = {'results_all': results_all, 'total_result_dict': total_result_dict, 'e1': e1, 'e2': e2} # 'gts_all': gts_all, 'inputs_all': inputs_all, 
    savepkl(results_dict, save_path)

MB_train_fit3d_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_tr_s03
Loading checkpoint checkpoint/pose3d/MB_train_fit3d_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_tr_s03/best_epoch.bin
/home/hrai/codes/MotionBERT/data/motion3d/yaml_files/FIT3D-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_REVOLUTE-TR_S03.yaml
Loading dataset...
FIT3D-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_REVOLUTE-TR_S03
[overwrite: False] ==> Loading FIT3D source_list...
[overwrite: False] ==> Loading FIT3D cam_param...
[overwrite: False] ==> Loading FIT3D cam_3d...
[overwrite: False] ==> Loading FIT3D cam_3d_canonical_revolute...
[overwrite: False] ==> Loading FIT3D img_2d_canonical_revolute...
INFO: Testing
No epoch information in the checkpoint


100%|██████████| 149/149 [00:51<00:00,  2.87it/s]


No eval_keypoint. Use part list
Part: whole
Protocol #1 Error (MPJPE): 40.90921648677834 mm
Protocol #2 Error (P-MPJPE): 25.34573916341777 mm
----------------------------------------


In [33]:
update_result_dict_pkl()

Loading result_dict.pkl


  0%|          | 0/88 [00:00<?, ?it/s]

100%|██████████| 88/88 [00:00<00:00, 199.06it/s]


In [32]:
# update result_dict
remove_subset_from_result_dict = True
if remove_subset_from_result_dict:
    result_dict = readpkl('result_dict.pkl')
    try: del result_dict[checkpoint_name][subset]
    except: print('subset not found')
    savepkl(result_dict, 'result_dict.pkl')

subset not found


### Evaluate and save all experiments

In [7]:
result_list = os.listdir('saved_results')
checkpoint_list = os.listdir('checkpoint/pose3d')
whitelist_checkpoint = []
for item in os.listdir('configs/pose3d'):
    checkpoint = item.split('.yaml')[0]
    if 'cam_no_factor' not in checkpoint: continue
    if checkpoint in blacklist_checkpoint: continue
    if checkpoint not in checkpoint_list:
        print(f'[Need to be trained] {checkpoint}')
        continue
    if checkpoint not in experiment_list:
        whitelist_checkpoint.append(checkpoint)
        # create empty file
        with open(f'experiments/{checkpoint}.txt', 'w') as f:
            pass
    else:
        subset_list = []
        with open(f'experiments/{checkpoint}.txt', 'r') as f:
            lines = f.readlines()
            for line in lines:
                subset = line.split('\n')[0]
                subset_list.append(subset)
        if len(subset_list) == 0:
            print(f'[No subset fou  d] {checkpoint}')
            whitelist_checkpoint.append(checkpoint)
        for subset in subset_list:
            if not os.path.exists(f'saved_results/{checkpoint}/{subset}.pkl'):
                print(f'[Result not found] {checkpoint} - {subset}')
                if checkpoint not in whitelist_checkpoint:
                    whitelist_checkpoint.append(checkpoint)

print('\nCheckpoints to evaluate:')
whitelist_checkpoint

[Need to be trained] MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_same_z_input_centering_steprot_1_tr_s1_ts_s5678
[No subset fou  d] MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute
[Need to be trained] MB_train_fit3d_gt_cam_no_factor_input_from_canonical_3d_same_z_input_centering_ts_s4710
[Need to be trained] MB_train_fit3d_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_ts_s4710_seed3
[Need to be trained] MB_train_h36m_gt_cam_no_factor_steprot_0.5_sinu_pitch_m15_p273_sinu_roll_m15_p273
[Need to be trained] MB_train_h36m_gt_cam_no_factor_no_rootrel
[Need to be trained] MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_same_z_input_centering_steprot_1_tr_s1_ts_s5678_no_rootrel
[Need to be trained] MB_train_fit3d_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_ts_s4710_seed1
[Need to be trained] MB_train_h36m_gt_cam_no_factor_input_from_canonical_pcl_input_centering_seed1
[Need to be trained] MB_train_h36m_gt_cam_no_facto

['MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute',
 'MB_train_fit3d_gt_cam_no_factor_tr_s03_seed4']

In [11]:
whitelist_checkpoint = [
## fit3d_ts_s4710
    # 'MB_train_fit3d_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_ts_s4710',
    # 'MB_train_fit3d_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_ts_s4710_seed1',
    # 'MB_train_fit3d_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_ts_s4710_seed2',
    # 'MB_train_fit3d_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_ts_s4710_seed3',
    # 'MB_train_fit3d_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_ts_s4710_seed4',
    ##
    # 'MB_train_fit3d_gt_cam_no_factor_input_from_canonical_pcl_original_ts_s4710',
    # 'MB_train_fit3d_gt_cam_no_factor_input_from_canonical_pcl_original_ts_s4710_seed1',
    # 'MB_train_fit3d_gt_cam_no_factor_input_from_canonical_pcl_original_ts_s4710_seed2',
    # 'MB_train_fit3d_gt_cam_no_factor_input_from_canonical_pcl_original_ts_s4710_seed3',
    # 'MB_train_fit3d_gt_cam_no_factor_input_from_canonical_pcl_original_ts_s4710_seed4',
    ##
    # 'MB_train_fit3d_gt_cam_no_factor_input_centering_ts_s4710',
    # 'MB_train_fit3d_gt_cam_no_factor_input_centering_ts_s4710_seed1',
    # 'MB_train_fit3d_gt_cam_no_factor_input_centering_ts_s4710_seed2',
    # 'MB_train_fit3d_gt_cam_no_factor_input_centering_ts_s4710_seed3',
    # 'MB_train_fit3d_gt_cam_no_factor_input_centering_ts_s4710_seed4',
    ##
    # 'MB_train_fit3d_gt_cam_no_factor_ts_s4710',
    # 'MB_train_fit3d_gt_cam_no_factor_ts_s4710_seed1',
    # 'MB_train_fit3d_gt_cam_no_factor_ts_s4710_seed2',
    # 'MB_train_fit3d_gt_cam_no_factor_ts_s4710_seed3',
    # 'MB_train_fit3d_gt_cam_no_factor_ts_s4710_seed4',
## fit3d_tr_s03
    # 'MB_train_fit3d_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_tr_s03',
    # 'MB_train_fit3d_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_tr_s03_seed1',
    # 'MB_train_fit3d_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_tr_s03_seed2',
    # 'MB_train_fit3d_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_tr_s03_seed3',
    # 'MB_train_fit3d_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_tr_s03_seed4',
    ##
    # 'MB_train_fit3d_gt_cam_no_factor_input_from_canonical_pcl_original_tr_s03',
    # 'MB_train_fit3d_gt_cam_no_factor_input_from_canonical_pcl_original_tr_s03_seed1',
    # 'MB_train_fit3d_gt_cam_no_factor_input_from_canonical_pcl_original_tr_s03_seed2',
    # 'MB_train_fit3d_gt_cam_no_factor_input_from_canonical_pcl_original_tr_s03_seed3',
    # 'MB_train_fit3d_gt_cam_no_factor_input_from_canonical_pcl_original_tr_s03_seed4',
    ##
    # 'MB_train_fit3d_gt_cam_no_factor_input_centering_tr_s03',
    # 'MB_train_fit3d_gt_cam_no_factor_input_centering_tr_s03_seed1',
    # 'MB_train_fit3d_gt_cam_no_factor_input_centering_tr_s03_seed2',
    # 'MB_train_fit3d_gt_cam_no_factor_input_centering_tr_s03_seed3',
    # 'MB_train_fit3d_gt_cam_no_factor_input_centering_tr_s03_seed4',
    ##
    # 'MB_train_fit3d_gt_cam_no_factor_tr_s03',
    # 'MB_train_fit3d_gt_cam_no_factor_tr_s03_seed1',
    # 'MB_train_fit3d_gt_cam_no_factor_tr_s03_seed2',
    # 'MB_train_fit3d_gt_cam_no_factor_tr_s03_seed3',
    # 'MB_train_fit3d_gt_cam_no_factor_tr_s03_seed4',
## 3dhp_tr_s1
    # 'MB_train_3dhp_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_tr_s1',
    # 'MB_train_3dhp_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_tr_s1_seed1',
    # 'MB_train_3dhp_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_tr_s1_seed2',
    # 'MB_train_3dhp_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_tr_s1_seed3',
    # 'MB_train_3dhp_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_tr_s1_seed4',
    ##
    # 'MB_train_3dhp_gt_cam_no_factor_input_from_canonical_pcl_original_tr_s1',
    # 'MB_train_3dhp_gt_cam_no_factor_input_from_canonical_pcl_original_tr_s1_seed1',
    # 'MB_train_3dhp_gt_cam_no_factor_input_from_canonical_pcl_original_tr_s1_seed2',
    # 'MB_train_3dhp_gt_cam_no_factor_input_from_canonical_pcl_original_tr_s1_seed3',
    # 'MB_train_3dhp_gt_cam_no_factor_input_from_canonical_pcl_original_tr_s1_seed4',
    ## 
    # 'MB_train_3dhp_gt_cam_no_factor_input_centering_tr_s1',
    # 'MB_train_3dhp_gt_cam_no_factor_input_centering_tr_s1_seed1',
    # 'MB_train_3dhp_gt_cam_no_factor_input_centering_tr_s1_seed2',
    # 'MB_train_3dhp_gt_cam_no_factor_input_centering_tr_s1_seed3',
    # 'MB_train_3dhp_gt_cam_no_factor_input_centering_tr_s1_seed4',
    ##
    # 'MB_train_3dhp_gt_cam_no_factor_tr_s1',
    # 'MB_train_3dhp_gt_cam_no_factor_tr_s1_seed1',
    # 'MB_train_3dhp_gt_cam_no_factor_tr_s1_seed2',
    # 'MB_train_3dhp_gt_cam_no_factor_tr_s1_seed3',
    # 'MB_train_3dhp_gt_cam_no_factor_tr_s1_seed4',
## 3dhp train
    # 'MB_train_3dhp_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering',
    # 'MB_train_3dhp_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_seed1',
    # 'MB_train_3dhp_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_seed2',
    # 'MB_train_3dhp_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_seed3',
    # 'MB_train_3dhp_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_seed4',
    ##
    # 'MB_train_3dhp_gt_cam_no_factor_input_centering',
    # 'MB_train_3dhp_gt_cam_no_factor_input_centering_seed1',
    # 'MB_train_3dhp_gt_cam_no_factor_input_centering_seed2',
    # 'MB_train_3dhp_gt_cam_no_factor_input_centering_seed3',
    # 'MB_train_3dhp_gt_cam_no_factor_input_centering_seed4',
    ##
    # 'MB_train_3dhp_gt_cam_no_factor',
    # 'MB_train_3dhp_gt_cam_no_factor_seed1',
    # 'MB_train_3dhp_gt_cam_no_factor_seed2',
    # 'MB_train_3dhp_gt_cam_no_factor_seed3',
    # 'MB_train_3dhp_gt_cam_no_factor_seed4',
## h36m train
    # 'MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_no_Rz_input_centering_seed1',
    # 'MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_no_Rz_input_centering_seed2',
    # 'MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_no_Rz_input_centering_seed3',
    # 'MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_no_Rz_input_centering_seed4',
    ##
    # 'MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_with_Kvirt',
    # 'MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_with_Kvirt_seed1',
    # 'MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_with_Kvirt_seed2',
    # 'MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_with_Kvirt_seed3',
    # 'MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_with_Kvirt_seed4',
    ##
    # 'MB_train_h36m_gt_cam_no_factor_input_from_canonical_pcl_with_Rz_input_centering',
    # 'MB_train_h36m_gt_cam_no_factor_input_from_canonical_pcl_with_Rz_input_centering_seed1',
    # 'MB_train_h36m_gt_cam_no_factor_input_from_canonical_pcl_with_Rz_input_centering_seed2',
    # 'MB_train_h36m_gt_cam_no_factor_input_from_canonical_pcl_with_Rz_input_centering_seed3',
    # 'MB_train_h36m_gt_cam_no_factor_input_from_canonical_pcl_with_Rz_input_centering_seed4',
    ##
    'MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_test_no_2d_canon',
    'MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_test_no_2d_canon_seed1',
    'MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_test_no_2d_canon_seed2',
    'MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_test_no_2d_canon_seed3',
    'MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_test_no_2d_canon_seed4',
    ##
    'MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_test_no_2d_canon_with_fix_orientation_pred',
    'MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_test_no_2d_canon_with_fix_orientation_pred_seed1',
    'MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_test_no_2d_canon_with_fix_orientation_pred_seed2',
    'MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_test_no_2d_canon_with_fix_orientation_pred_seed3',
    'MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_test_no_2d_canon_with_fix_orientation_pred_seed4',
    ##
    # 'MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering',
    # 'MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_seed1',
    # 'MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_seed2',
    # 'MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_seed3',
    # 'MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_seed4',
    ##
    # 'MB_train_h36m_gt_cam_no_factor_input_centering',
    # 'MB_train_h36m_gt_cam_no_factor_input_centering_seed1',
    # 'MB_train_h36m_gt_cam_no_factor_input_centering_seed2',
    # 'MB_train_h36m_gt_cam_no_factor_input_centering_seed3',
    # 'MB_train_h36m_gt_cam_no_factor_input_centering_seed4',
    ##
    # 'MB_train_h36m_gt_cam_no_factor',
    # 'MB_train_h36m_gt_cam_no_factor_seed1',
    # 'MB_train_h36m_gt_cam_no_factor_seed2',
    # 'MB_train_h36m_gt_cam_no_factor_seed3',
    # 'MB_train_h36m_gt_cam_no_factor_seed4',
## h36m_tr_s1_ts_s5678
    # 'MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_tr_s1_ts_s5678',
    # 'MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_tr_s1_ts_s5678_seed1',
    # 'MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_tr_s1_ts_s5678_seed2',
    # 'MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_tr_s1_ts_s5678_seed3',
    # 'MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_tr_s1_ts_s5678_seed4',
    ##
    # 'MB_train_h36m_gt_cam_no_factor_input_centering_tr_s1_ts_s5678',
    # 'MB_train_h36m_gt_cam_no_factor_input_centering_tr_s1_ts_s5678_seed1',
    # 'MB_train_h36m_gt_cam_no_factor_input_centering_tr_s1_ts_s5678_seed2',
    # 'MB_train_h36m_gt_cam_no_factor_input_centering_tr_s1_ts_s5678_seed3',
    # 'MB_train_h36m_gt_cam_no_factor_input_centering_tr_s1_ts_s5678_seed4',
    ##
    # 'MB_train_h36m_gt_cam_no_factor_tr_s1_ts_s5678',
    # 'MB_train_h36m_gt_cam_no_factor_tr_s1_ts_s5678_seed1',
    # 'MB_train_h36m_gt_cam_no_factor_tr_s1_ts_s5678_seed2',
    # 'MB_train_h36m_gt_cam_no_factor_tr_s1_ts_s5678_seed3',
    # 'MB_train_h36m_gt_cam_no_factor_tr_s1_ts_s5678_seed4',
## h36m_tr_s15_ts_s678
    # 'MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_tr_s15_ts_s678',
    # 'MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_tr_s15_ts_s678_seed1',
    # 'MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_tr_s15_ts_s678_seed2',
    # 'MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_tr_s15_ts_s678_seed3',
    # 'MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_tr_s15_ts_s678_seed4',
    # #
    # 'MB_train_h36m_gt_cam_no_factor_input_centering_tr_s15_ts_s678',
    # 'MB_train_h36m_gt_cam_no_factor_input_centering_tr_s15_ts_s678_seed1',
    # 'MB_train_h36m_gt_cam_no_factor_input_centering_tr_s15_ts_s678_seed2',
    # 'MB_train_h36m_gt_cam_no_factor_input_centering_tr_s15_ts_s678_seed3',
    # 'MB_train_h36m_gt_cam_no_factor_input_centering_tr_s15_ts_s678_seed4',
    # #
    # 'MB_train_h36m_gt_cam_no_factor_tr_s15_ts_s678',
    # 'MB_train_h36m_gt_cam_no_factor_tr_s15_ts_s678_seed1',
    # 'MB_train_h36m_gt_cam_no_factor_tr_s15_ts_s678_seed2',
    # 'MB_train_h36m_gt_cam_no_factor_tr_s15_ts_s678_seed3',
    # 'MB_train_h36m_gt_cam_no_factor_tr_s15_ts_s678_seed4',
]

# check experiment file
check_config_experiment(whitelist_checkpoint)

MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_test_no_2d_canon
Config# path: /home/hrai/codes/MotionBERT/configs/pose3d/MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_test_no_2d_canon.yaml
Experiment path: /home/hrai/codes/MotionBERT/experiments/MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_test_no_2d_canon.txt
MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_test_no_2d_canon_seed1
Config# path: /home/hrai/codes/MotionBERT/configs/pose3d/MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_test_no_2d_canon_seed1.yaml
Experiment path: /home/hrai/codes/MotionBERT/experiments/MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_test_no_2d_canon_seed1.txt
MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_test_no_2d_canon_seed2
Config# path: /home/hrai/codes/MotionBERT/configs/p

In [12]:
experiment_root = f'/home/{user}/codes/MotionBERT/experiments'
for item in glob(experiment_root+'/*.txt'):
    checkpoint_name = os.path.basename(item).split('.txt')[0]
    # check if in whitelist
    if checkpoint_name in blacklist_checkpoint: continue
    if len(whitelist_checkpoint) > 0 and checkpoint_name not in whitelist_checkpoint: continue
    # filtering
    if 'cam_no_factor' not in checkpoint_name: continue
    # get save folder
    save_folder = f'/home/{user}/codes/MotionBERT/saved_results/{checkpoint_name}'
    if not os.path.exists(save_folder): os.makedirs(save_folder)

    # load model
    args, opts = get_opt_args_from_model_name(checkpoint_name, verbose=False)
    try:
        model_pos, chk_filename, checkpoint = load_model(opts, args)
    except:
        print(f'Error loading model: {checkpoint_name}')
        continue

    # load subset list
    subset_list = []
    with open(item, 'r') as f:
        lines = f.readlines()
        for line in lines:
            subset = line.strip()
            subset_list.append(subset)

    for subset in subset_list:
        if 'PCL' in subset: args.fix_orientation_pred = True
        # get save path
        save_path = os.path.join(save_folder, f'{subset}.pkl')
        # check if exists
        if os.path.exists(save_path):
            print(f'{save_path} exists')
            continue
        # evaluation
        try:
            # load dataset
            args.subset_list = [subset]
            train_loader, test_loader, _, _, datareader = load_dataset(args, use_new_datareader=True)
            # evaluation
            e1, e2, results_all, inputs_all, gts_all, total_result_dict = evaluate(args, model_pos, test_loader, datareader, checkpoint, only_one_batch=False)
            # save results
            results_dict = {'results_all': results_all, 'total_result_dict': total_result_dict, 'e1': e1, 'e2': e2} # 'gts_all': gts_all, 'inputs_all': inputs_all, 
            savepkl(results_dict, save_path)
        except Exception as e:
            print(f'Error: {e}')


MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_test_no_2d_canon_seed1
Loading checkpoint checkpoint/pose3d_h36m/MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_test_no_2d_canon_seed1/best_epoch.bin
Loading dataset...
H36M-GT-CAM_NO_FACTOR
[overwrite: False] ==> Loading H36M source_list...
[overwrite: False] ==> Loading H36M cam_param...
[overwrite: False] ==> Loading H36M cam_3d...
[overwrite: False] ==> Loading H36M img_2d...
INFO: Testing
No epoch information in the checkpoint


100%|██████████| 133/133 [00:45<00:00,  2.90it/s]


in
in3
No eval_keypoint. Use part list
Part: whole
Protocol #1 Error (MPJPE): 48.609439330889586 mm
Protocol #2 Error (P-MPJPE): 16.920884629639584 mm
----------------------------------------
Loading dataset...
FIT3D-GT-CAM_NO_FACTOR-ALL_TEST
[overwrite: False] ==> Loading FIT3D source_list...
[overwrite: False] ==> Loading FIT3D cam_param...
[overwrite: False] ==> Loading FIT3D cam_3d...
[overwrite: False] ==> Loading FIT3D img_2d...
INFO: Testing
No epoch information in the checkpoint


100%|██████████| 411/411 [02:19<00:00,  2.94it/s]


in
in3
No eval_keypoint. Use part list
Part: whole
Protocol #1 Error (MPJPE): 93.42098244680129 mm
Protocol #2 Error (P-MPJPE): 71.03992183469 mm
----------------------------------------
Loading dataset...
3DHP-GT-CAM_NO_FACTOR-TEST_TS1_6
[overwrite: False] ==> Loading 3DHP source_list...
[overwrite: False] ==> Loading 3DHP cam_param...
[overwrite: False] ==> Loading 3DHP cam_3d...
[overwrite: False] ==> Loading 3DHP img_2d...
INFO: Testing
No epoch information in the checkpoint


...

in
in3
No eval_keypoint. Use part list
Part: whole
Protocol #1 Error (MPJPE): 82.287532627265 mm
Protocol #2 Error (P-MPJPE): 56.39507174022797 mm
----------------------------------------
Loading dataset...
3DHP-GT-CAM_NO_FACTOR-TEST_TS1_6_UNIV
[overwrite: False] ==> Loading 3DHP source_list...
[overwrite: False] ==> Loading 3DHP cam_param...
[overwrite: False] ==> Loading 3DHP cam_3d...
[overwrite: False] ==> Loading 3DHP img_2d...
INFO: Testing
No epoch information in the checkpoint


100%|██████████| 6/6 [00:02<00:00,  2.14it/s]


in
in3
No eval_keypoint. Use part list
Part: whole
Protocol #1 Error (MPJPE): 64.75390565054586 mm
Protocol #2 Error (P-MPJPE): 50.329991815024435 mm
----------------------------------------
Loading dataset...
3DHP-GT-CAM_NO_FACTOR-TEST_ALL_TRAIN
[overwrite: False] ==> Loading 3DHP source_list...
[overwrite: False] ==> Loading 3DHP cam_param...
[overwrite: False] ==> Loading 3DHP cam_3d...
[overwrite: False] ==> Loading 3DHP img_2d...
INFO: Testing
No epoch information in the checkpoint


100%|██████████| 245/245 [01:23<00:00,  2.92it/s]


in
in3
No eval_keypoint. Use part list
Part: whole
Protocol #1 Error (MPJPE): 71.22894793713598 mm
Protocol #2 Error (P-MPJPE): 56.76726453556982 mm
----------------------------------------
MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_test_no_2d_canon_with_fix_orientation_pred
Loading checkpoint checkpoint/pose3d_h36m/MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_test_no_2d_canon_with_fix_orientation_pred/best_epoch.bin
/home/hrai/codes/MotionBERT/saved_results/MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_test_no_2d_canon_with_fix_orientation_pred/H36M-GT-CAM_NO_FACTOR.pkl exists
/home/hrai/codes/MotionBERT/saved_results/MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_test_no_2d_canon_with_fix_orientation_pred/FIT3D-GT-CAM_NO_FACTOR-ALL_TEST.pkl exists
/home/hrai/codes/MotionBERT/saved_results/MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revo

In [18]:
args.mpjpe_mode

'cam_3d_from_canonical_3d'

In [10]:
datareader.datareader['3DHP-GT-CAM_NO_FACTOR-TEST_TS1_6'].data_type_list

['cam_3d',
 'joint_2d',
 'source',
 'cam_param',
 'camera_name',
 'action',
 'confidence']

### Reset the result_dict for whitelist_checkpoint

In [4]:
result_dict = readpkl('result_dict.pkl')
for item in whitelist_checkpoint:
    try: del result_dict[item]
    except: print(f'[{item}] not found')
savepkl(result_dict, 'result_dict.pkl')

In [8]:
savepkl(result_dict, 'result_dict.pkl')

### Cherrypick e1, e2 -> result_dict.pkl

In [13]:
blacklist_checkpoint = ['MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_same_z_s15678_tr_54138969_ts_others']
update_result_dict_pkl(blacklist_checkpoint)

Loading result_dict.pkl


  0%|          | 0/146 [00:00<?, ?it/s]

/home/hrai/codes/MotionBERT/saved_results/MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_test_no_2d_canon_with_fix_orientation_pred_seed1/3DHP-GT-CAM_NO_FACTOR-TEST_ALL_TRAIN.pkl
/home/hrai/codes/MotionBERT/saved_results/MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_test_no_2d_canon_with_fix_orientation_pred_seed1/3DHP-GT-CAM_NO_FACTOR-TEST_TS1_6.pkl
/home/hrai/codes/MotionBERT/saved_results/MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_test_no_2d_canon_with_fix_orientation_pred_seed1/FIT3D-GT-CAM_NO_FACTOR-ALL_TEST.pkl
/home/hrai/codes/MotionBERT/saved_results/MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_test_no_2d_canon_with_fix_orientation_pred_seed1/H36M-GT-CAM_NO_FACTOR.pkl


  5%|▍         | 7/146 [00:01<00:35,  3.96it/s]

/home/hrai/codes/MotionBERT/saved_results/MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_test_no_2d_canon_with_fix_orientation_pred_seed1/3DHP-GT-CAM_NO_FACTOR-TEST_TS1_6_UNIV.pkl
/home/hrai/codes/MotionBERT/saved_results/MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_test_no_2d_canon_seed1/3DHP-GT-CAM_NO_FACTOR-TEST_ALL_TRAIN.pkl
/home/hrai/codes/MotionBERT/saved_results/MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_test_no_2d_canon_seed1/3DHP-GT-CAM_NO_FACTOR-TEST_TS1_6.pkl
/home/hrai/codes/MotionBERT/saved_results/MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_test_no_2d_canon_seed1/FIT3D-GT-CAM_NO_FACTOR-ALL_TEST.pkl
/home/hrai/codes/MotionBERT/saved_results/MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_test_no_2d_canon_seed1/H36M-GT-CAM_NO_FACTOR.pkl


...

/home/hrai/codes/MotionBERT/saved_results/MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_test_no_2d_canon_seed3/3DHP-GT-CAM_NO_FACTOR-TEST_TS1_6_UNIV.pkl
/home/hrai/codes/MotionBERT/saved_results/MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_test_no_2d_canon_with_fix_orientation_pred_seed3/3DHP-GT-CAM_NO_FACTOR-TEST_ALL_TRAIN.pkl
/home/hrai/codes/MotionBERT/saved_results/MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_test_no_2d_canon_with_fix_orientation_pred_seed3/3DHP-GT-CAM_NO_FACTOR-TEST_TS1_6.pkl
/home/hrai/codes/MotionBERT/saved_results/MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_test_no_2d_canon_with_fix_orientation_pred_seed3/FIT3D-GT-CAM_NO_FACTOR-ALL_TEST.pkl
/home/hrai/codes/MotionBERT/saved_results/MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_test_no_2d_canon_with_fix_orientation_pred_seed3/H36M-GT-CAM_NO_

 92%|█████████▏| 135/146 [00:13<00:01,  8.98it/s]

/home/hrai/codes/MotionBERT/saved_results/MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_test_no_2d_canon_with_fix_orientation_pred_seed3/3DHP-GT-CAM_NO_FACTOR-TEST_TS1_6_UNIV.pkl
/home/hrai/codes/MotionBERT/saved_results/MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_test_no_2d_canon_with_fix_orientation_pred_seed4/3DHP-GT-CAM_NO_FACTOR-TEST_ALL_TRAIN.pkl
/home/hrai/codes/MotionBERT/saved_results/MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_test_no_2d_canon_with_fix_orientation_pred_seed4/3DHP-GT-CAM_NO_FACTOR-TEST_TS1_6.pkl
/home/hrai/codes/MotionBERT/saved_results/MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_test_no_2d_canon_with_fix_orientation_pred_seed4/FIT3D-GT-CAM_NO_FACTOR-ALL_TEST.pkl
/home/hrai/codes/MotionBERT/saved_results/MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_test_no_2d_canon_with_fix_orientation_

100%|██████████| 146/146 [00:15<00:00,  9.41it/s]

/home/hrai/codes/MotionBERT/saved_results/MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_test_no_2d_canon_with_fix_orientation_pred_seed4/3DHP-GT-CAM_NO_FACTOR-TEST_TS1_6_UNIV.pkl


### Calculate PCK, AUC for 3DHP

In [ ]:
def compute_PCK(gts, preds, scales=1000, eval_joints=None, threshold=150):
    PCK_THRESHOLD = threshold
    sample_num = len(gts)
    total = 0
    true_positive = 0
    if eval_joints is None:
        eval_joints = list(range(gts.shape[1]))

    for n in range(sample_num):
        gt = gts[n]
        pred = preds[n]
        # scale = scales[n]
        scale = 1000
        per_joint_error = np.take(np.sqrt(np.sum(np.power(pred - gt, 2), 1)) * scale, eval_joints, axis=0)
        true_positive += (per_joint_error < PCK_THRESHOLD).sum()
        total += per_joint_error.size

    pck = float(true_positive / total) * 100
    return pck


def compute_AUC(gts, preds, scales=1000, eval_joints=None):
    # This range of thresholds mimics 'mpii_compute_3d_pck.m', which is provided as part of the
    # MPI-INF-3DHP test data release.
    thresholds = np.linspace(0, 150, 31)
    pck_list = []
    for threshold in thresholds:
        pck_list.append(compute_PCK(gts, preds, scales, eval_joints, threshold))

    auc = np.mean(pck_list)

    return auc

In [7]:
whitelist_checkpoint = [
    'MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering',
]

# check experiment file
check_config_experiment(whitelist_checkpoint)

MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering
Experiment path: /home/hrai/codes/MotionBERT/experiments/MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering.txt
Config# path: /home/hrai/codes/MotionBERT/configs/pose3d/MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering.yaml


In [27]:
user = getpass.getuser()
result_root = f'/home/{user}/codes/MotionBERT/saved_results'
for checkpoint_name in tqdm(whitelist_checkpoint):
    checkpoint_root = os.path.join(result_root, checkpoint_name)
    subset_list = glob(checkpoint_root+'/*.pkl')
    print(subset_list)
    for item in subset_list:
        if 'TS1_6_UNIV' not in item: continue
        print(item)
        args, opts = get_opt_args_from_model_name(checkpoint_name, verbose=False)
        args.subset_list = [os.path.basename(item).split('.pkl')[0]]
        _, test_loader, _, _, datareader = load_dataset(args, use_new_datareader=True)
        subset = os.path.basename(item).split('.pkl')[0]
        result = readpkl(item)
        print(result.keys())

        # get gts_all
        gts_all = []
        with torch.no_grad():
            for batch_input, batch_gt in tqdm(test_loader): # batch_input: normalized joint_2d, batch_gt: normalized joint3d_image
                # preprocessing
                batch_gt_original = batch_gt.clone().detach().cuda()
                batch_input, batch_gt, batch_gt_torso, batch_gt_limb_pos = preprocess_eval(args, batch_input, batch_gt)
                gts_all.append(batch_gt.cpu().numpy())
        gts_all = np.concatenate(gts_all)
        if args.denormalize_output:
            gts_all = datareader.denormalize(gts_all) # denormalize the ground truth 3D poses

        results_all = result['results_all'].copy().reshape(-1, 17, 3)
        gts_all = gts_all.reshape(-1, 17, 3)
        
        pck = compute_PCK(gts_all, results_all)
        auc = compute_AUC(gts_all, results_all)
        print(f'PCK: {pck}')
        print(f'AUC: {auc}')
        #break
    #break

  0%|          | 0/1 [00:00<?, ?it/s]

['/home/hrai/codes/MotionBERT/saved_results/MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering/3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_CANONICAL_PCL_WITH_RZ-TEST_TS1_6_UNIV.pkl', '/home/hrai/codes/MotionBERT/saved_results/MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering/H36M-GT-CAM_NO_FACTOR-INPUT_FROM_CANONICAL_PCL_WITH_RZ.pkl', '/home/hrai/codes/MotionBERT/saved_results/MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering/3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_REVOLUTE-TEST_TS1_6.pkl', '/home/hrai/codes/MotionBERT/saved_results/MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering/H36M-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_REVOLUTE.pkl', '/home/hrai/codes/MotionBERT/saved_results/MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering/3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_CANONICAL_PCL_WITH_RZ-TEST_TS1_6.pkl', '/home/hrai/codes/MotionBERT/saved_result

100%|██████████| 6/6 [00:00<00:00, 11.81it/s]


PCK: 90.39000511041179
AUC: 62.87428126700035
/home/hrai/codes/MotionBERT/saved_results/MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering/3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_REVOLUTE-TEST_TS1_6_UNIV.pkl
Loading dataset...
3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_REVOLUTE-TEST_TS1_6_UNIV
[overwrite: False] ==> Loading 3DHP source_list...
[overwrite: False] ==> Loading 3DHP cam_param...
[overwrite: False] ==> Loading 3DHP cam_3d...
[overwrite: False] ==> Loading 3DHP cam_3d_canonical_revolute...
[overwrite: False] ==> Loading 3DHP img_2d_canonical_revolute...
dict_keys(['results_all', 'total_result_dict', 'e1', 'e2'])


100%|██████████| 1/1 [00:32<00:00, 32.12s/it]

PCK: 90.39000511041179
AUC: 62.874298619841746


In [19]:
results_all.shape, gts_all.shape

((21870, 17, 3), (21870, 17, 3))

In [25]:
pck = compute_PCK(gts_all, results_all)
auc = compute_AUC(gts_all, results_all)

In [26]:
pck, auc

(90.39000511041179, 62.87428126700035)

### Delete gts_all, inputs_all to reduce the file size

In [ ]:
experiment_root = f'/home/{user}/codes/MotionBERT/experiments'
for item in tqdm(glob(experiment_root+'/*.txt')):
    checkpoint_name = os.path.basename(item).split('.txt')[0]
    save_folder = f'/home/hrai/codes/MotionBERT/saved_results/{checkpoint_name}'
    subset_list = []
    with open(item, 'r') as f:
        lines = f.readlines()
        for line in lines:
            subset = line.strip()
            subset_list.append(subset)

    for subset in subset_list:
        save_path = os.path.join(save_folder, f'{subset}.pkl')
        if not os.path.exists(save_path): continue
        results_dict = readpkl(save_path)
        #print(checkpoint_name, subset, results_dict.keys())
        if 'gts_all' in results_dict.keys(): del results_dict['gts_all']
        if 'inputs_all' in results_dict.keys(): del results_dict['inputs_all']
        savepkl(results_dict, save_path)

  0%|          | 0/97 [00:00<?, ?it/s]

 31%|███       | 30/97 [10:41<22:45, 20.39s/it]